In [2]:
import os
import pandas as pd
import numpy as np
from ast import literal_eval

## Basic functions to manipulate a dataframe

In [3]:
def path_to_files():
    '''
    Printing out the files in a directory of choice for convenience.
    '''
    path = input('Enter a path to the files: ')
    files_path = os.scandir(path)

    for file in files_path:
        if file.is_dir() or file.is_file():
            print(file.name)

In [4]:
def open_dataframe(file_path, file_name):
    '''
    Opens a CSV file and sets the maximum available columns to be shown.
    Requires path to file and name of the file.
    Returns a dataframe of the file.
    '''
    df = pd.read_csv(file_path + '/' + file_name)
    pd.set_option('display.max_columns', None)
    return df

In [5]:
def inspect_dataframe(dataframe):
    '''
    Returns information about the dataset 
    and shows a preview of the data in it.
    '''
    print(dataframe.info(), '\n')
    print(dataframe.head())

In [6]:
def list_like_columns(dataframe):
    '''
    Creating a list where columns have list-like values.
    '''
    list_like_cols = []

    for row in dataframe:
        col_name = row
        empty_list = dataframe[dataframe.astype(str)[row] == '[]'].index
        if len(empty_list) != 0:
            list_like_cols.append(col_name)
    
    return list_like_cols

In [7]:
def to_proper_list(dataframe, list_like_columns):
    '''
    Converting values of list-like columns to proper list type.
    '''
    for row in list_like_columns:
        dataframe[row] = dataframe[row].apply(literal_eval)
    
    return dataframe

In [8]:
def copy_and_explode(dataframe, *columns):
    '''
    Normalizing to the 2NF.
    Creating new dataframes containing str-list columns.
    Exploding list values of columns.
    Recreating the index column if the original dataset had one
    to keep unique index in the dataframe.
    Saving new dataframe to a CSV file.
    '''
    df_copy = dataframe.copy()
    col_list = [*columns]
    

    for col in col_list:
        if isinstance(col_list, (list, pd.core.series.Series, np.ndarray)):
            df_copy = df_copy.explode(col)
            df_copy[col].replace(',', '', regex=True, inplace=True)
            df_copy = df_copy.explode(col)
        else:
            df_copy
    
    df_copy = df_copy.drop_duplicates()
       
    new_df_name = input('Enter a name for a normalized dataset: ')
    df_copy.to_csv(f'../data/output/{new_df_name}_2NF.csv', sep=',', encoding='utf-8', index=None, header='true')

In [9]:
def save_to_csv(dataframe):
    '''
    Save the dataframes as CSV files.
    Requires path to save location
    and name of the file to save as.
    '''
    path_to_save = input('Path to save location: ')
    name_of_file = input('File name: ')
    dataframe.to_csv(f'{path_to_save}/{name_of_file}.csv', sep=',', encoding='utf-8', index=None, header='true')

## Finding the longest value in each column of a given dataframe

In [10]:
def longest_value(dataframe):
    '''
    Checking for the longest string in each column of a given CSV file.
    '''
    for row in dataframe:
        col_name = row
        row_pos = dataframe[row].astype(str).str.len().idxmax(axis=1)

        if dataframe[row].dtype == float or int:
            list_len = dataframe[row].astype(str).str.len().max()

        else:
            list_len = max(list(map(len, dataframe[row].values)))

        print('Column:', col_name)
        print('Value length:', list_len,)
        print('Row possition:', row_pos, '\n')

## Files in ../data/input location

In [247]:
input_files = path_to_files()

Enter a path to the files: ../data/input
Best Movie by Year Netflix.csv
Best Movies Netflix.csv
Best Show by Year Netflix.csv
Best Shows Netflix.csv
raw_credits.csv
raw_titles.csv


## DF1: Best Movie by Year Netflix.csv

In [13]:
df = open_dataframe('../data/input', 'Best Movie by Year Netflix.csv')

In [14]:
df_info = inspect_dataframe(df)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49 entries, 0 to 48
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   index            49 non-null     int64  
 1   TITLE            49 non-null     object 
 2   RELEASE_YEAR     49 non-null     int64  
 3   SCORE            49 non-null     float64
 4   MAIN_GENRE       49 non-null     object 
 5   MAIN_PRODUCTION  49 non-null     object 
dtypes: float64(1), int64(2), object(3)
memory usage: 2.4+ KB
None 

   index                 TITLE  RELEASE_YEAR  SCORE MAIN_GENRE MAIN_PRODUCTION
0      0       White Christmas          1954    7.5    romance              US
1      1  The Guns of Navarone          1961    7.5        war              US
2      2          My Fair Lady          1964    7.8      drama              US
3      3      Bonnie and Clyde          1967    7.7      drama              US
4      4           Dirty Harry          1971    7.7   thriller    

In [15]:
# Checking if there are duplicate rows

df.duplicated().value_counts()

False    49
dtype: int64

In [16]:
# Dropping duplicate rows

df = df.drop_duplicates()

In [17]:
df['MAIN_PRODUCTION'].values

array(['US', 'US', 'US', 'US', 'US', 'US', 'GB', 'US', 'GB', 'US', 'US',
       'US', 'US', 'US', 'GB', 'US', 'US', 'US', 'US', 'US', 'US', 'US',
       'US', 'JP', 'US', 'US', 'DE', 'IN', 'US', 'US', 'IN', 'FR', 'IN',
       'IN', 'HK', 'IN', 'GB', 'US', 'US', 'US', 'IN', 'IN', 'IN', 'IN',
       'IN', 'US', 'GB', 'US', 'GB'], dtype=object)

In [18]:
# Finding the longest value in every column.

df_val_len = longest_value(df)

Column: index
Value length: 2
Row possition: 10 

Column: TITLE
Value length: 46
Row possition: 23 

Column: RELEASE_YEAR
Value length: 4
Row possition: 0 

Column: SCORE
Value length: 3
Row possition: 0 

Column: MAIN_GENRE
Value length: 11
Row possition: 46 

Column: MAIN_PRODUCTION
Value length: 2
Row possition: 0 



In [20]:
# Saving to the '../data/output' folder

df_csv = save_to_csv(df)

Path to save location: ../data/output
File name: best_movie_by_year_2nf


## DF2: Best Movies Netflix.csv

In [21]:
df2 = open_dataframe('../data/input', 'Best Movies Netflix.csv')

In [22]:
df2_info = inspect_dataframe(df2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 387 entries, 0 to 386
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   index            387 non-null    int64  
 1   TITLE            387 non-null    object 
 2   RELEASE_YEAR     387 non-null    int64  
 3   SCORE            387 non-null    float64
 4   NUMBER_OF_VOTES  387 non-null    int64  
 5   DURATION         387 non-null    int64  
 6   MAIN_GENRE       387 non-null    object 
 7   MAIN_PRODUCTION  387 non-null    object 
dtypes: float64(1), int64(4), object(3)
memory usage: 24.3+ KB
None 

   index                                     TITLE  RELEASE_YEAR  SCORE  \
0      0  David Attenborough: A Life on Our Planet          2020    9.0   
1      1                                 Inception          2010    8.8   
2      2                              Forrest Gump          1994    8.8   
3      3                                Anbe Sivam          2003  

In [23]:
df2.duplicated().value_counts()

False    387
dtype: int64

In [24]:
df2['MAIN_PRODUCTION'].unique()

array(['GB', 'US', 'IN', 'UA', 'CD', 'TR', 'ES', 'AU', 'JP', 'ZA', 'HK',
       'DE', 'KR', 'CA', 'BE', 'NO', 'NZ', 'MX', 'FR', 'MW', 'TH', 'AR',
       'PS', 'HU', 'IT', 'CN', 'PL', 'KH', 'IE', 'BR', 'XX', 'LT', 'NL',
       'DK', 'ID'], dtype=object)

In [25]:
df2_val_len = longest_value(df2)

Column: index
Value length: 3
Row possition: 100 

Column: TITLE
Value length: 104
Row possition: 121 

Column: RELEASE_YEAR
Value length: 4
Row possition: 0 

Column: SCORE
Value length: 3
Row possition: 0 

Column: NUMBER_OF_VOTES
Value length: 7
Row possition: 1 

Column: DURATION
Value length: 3
Row possition: 1 

Column: MAIN_GENRE
Value length: 11
Row possition: 0 

Column: MAIN_PRODUCTION
Value length: 2
Row possition: 0 



In [26]:
df2_csv = save_to_csv(df2)

Path to save location: ../data/output
File name: best_movies_2nf


## DF3: Best Show by Year Netflix.csv

In [27]:
df3 = open_dataframe('../data/input', 'Best Show by Year Netflix.csv')

In [28]:
df3_info = inspect_dataframe(df3)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31 entries, 0 to 30
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   index              31 non-null     int64  
 1   TITLE              31 non-null     object 
 2   RELEASE_YEAR       31 non-null     int64  
 3   SCORE              31 non-null     float64
 4   NUMBER_OF_SEASONS  31 non-null     int64  
 5   MAIN_GENRE         31 non-null     object 
 6   MAIN_PRODUCTION    31 non-null     object 
dtypes: float64(1), int64(3), object(3)
memory usage: 1.8+ KB
None 

   index                         TITLE  RELEASE_YEAR  SCORE  \
0      0  Monty Python's Flying Circus          1969    8.8   
1      1                  Knight Rider          1982    6.9   
2      2                      Seinfeld          1989    8.9   
3      3    Star Trek: Deep Space Nine          1993    8.1   
4      4       Neon Genesis Evangelion          1995    8.5   

   NUMBER_OF_SEASO

In [29]:
df3.duplicated().value_counts()

False    31
dtype: int64

In [30]:
df3['MAIN_PRODUCTION'].values

array(['GB', 'US', 'US', 'US', 'JP', 'US', 'JP', 'JP', 'US', 'CA', 'JP',
       'US', 'US', 'US', 'JP', 'US', 'US', 'US', 'GB', 'JP', 'GB', 'JP',
       'US', 'US', 'US', 'CA', 'US', 'IN', 'US', 'US', 'GB'], dtype=object)

In [31]:
df3_val_len = longest_value(df3)

Column: index
Value length: 2
Row possition: 10 

Column: TITLE
Value length: 28
Row possition: 0 

Column: RELEASE_YEAR
Value length: 4
Row possition: 0 

Column: SCORE
Value length: 3
Row possition: 0 

Column: NUMBER_OF_SEASONS
Value length: 2
Row possition: 5 

Column: MAIN_GENRE
Value length: 11
Row possition: 28 

Column: MAIN_PRODUCTION
Value length: 2
Row possition: 0 



In [32]:
df3_csv = save_to_csv(df3)

Path to save location: ../data/output
File name: best_show_by_year_2nf


## DF4: Best Shows Netflix.csv

In [33]:
df4 = open_dataframe('../data/input', 'Best Shows Netflix.csv')

In [34]:
df4_info = inspect_dataframe(df4)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 246 entries, 0 to 245
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   index              246 non-null    int64  
 1   TITLE              246 non-null    object 
 2   RELEASE_YEAR       246 non-null    int64  
 3   SCORE              246 non-null    float64
 4   NUMBER_OF_VOTES    246 non-null    int64  
 5   DURATION           246 non-null    int64  
 6   NUMBER_OF_SEASONS  246 non-null    int64  
 7   MAIN_GENRE         246 non-null    object 
 8   MAIN_PRODUCTION    246 non-null    object 
dtypes: float64(1), int64(5), object(3)
memory usage: 17.4+ KB
None 

   index                       TITLE  RELEASE_YEAR  SCORE  NUMBER_OF_VOTES  \
0      0                Breaking Bad          2008    9.5          1727694   
1      1  Avatar: The Last Airbender          2005    9.3           297336   
2      2                  Our Planet          2019    9.3       

In [35]:
df4.duplicated().value_counts()

False    246
dtype: int64

In [36]:
df4['MAIN_PRODUCTION'].values

array(['US', 'US', 'GB', 'IN', 'US', 'US', 'JP', 'JP', 'JP', 'US', 'JP',
       'GB', 'US', 'GB', 'US', 'US', 'US', 'US', 'JP', 'GB', 'CA', 'DE',
       'US', 'JP', 'US', 'JP', 'US', 'US', 'US', 'GB', 'JP', 'CA', 'US',
       'US', 'US', 'US', 'AU', 'KR', 'US', 'US', 'IN', 'GB', 'US', 'US',
       'GB', 'CA', 'JP', 'GB', 'GB', 'JP', 'US', 'US', 'IN', 'GB', 'GB',
       'DK', 'US', 'CA', 'US', 'JP', 'GB', 'TR', 'US', 'US', 'US', 'GB',
       'KR', 'GB', 'DE', 'US', 'JP', 'US', 'US', 'US', 'US', 'JP', 'US',
       'IL', 'GB', 'KR', 'ES', 'SE', 'US', 'JP', 'US', 'US', 'CA', 'US',
       'US', 'FR', 'US', 'US', 'US', 'US', 'KR', 'US', 'US', 'US', 'ES',
       'US', 'SE', 'JP', 'US', 'JP', 'US', 'US', 'GB', 'JP', 'US', 'GB',
       'US', 'GB', 'US', 'US', 'US', 'US', 'US', 'GB', 'US', 'US', 'US',
       'US', 'US', 'BE', 'US', 'BR', 'KR', 'JP', 'US', 'US', 'CA', 'US',
       'DE', 'US', 'US', 'GB', 'JP', 'US', 'JP', 'US', 'US', 'GB', 'JP',
       'US', 'US', 'TR', 'CA', 'NO', 'KR', 'US', 'I

In [37]:
df4_val_len = longest_value(df4)

Column: index
Value length: 3
Row possition: 100 

Column: TITLE
Value length: 48
Row possition: 120 

Column: RELEASE_YEAR
Value length: 4
Row possition: 0 

Column: SCORE
Value length: 3
Row possition: 0 

Column: NUMBER_OF_VOTES
Value length: 7
Row possition: 0 

Column: DURATION
Value length: 3
Row possition: 216 

Column: NUMBER_OF_SEASONS
Value length: 2
Row possition: 18 

Column: MAIN_GENRE
Value length: 11
Row possition: 2 

Column: MAIN_PRODUCTION
Value length: 2
Row possition: 0 



In [38]:
df4_csv = save_to_csv(df4)

Path to save location: ../data/output
File name: best_shows_2nf


## DF5: raw_credits.csv

In [118]:
df5 = open_dataframe('../data/input', 'raw_credits.csv')

In [119]:
df5_info = inspect_dataframe(df5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77213 entries, 0 to 77212
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   index      77213 non-null  int64 
 1   person_id  77213 non-null  int64 
 2   id         77213 non-null  object
 3   name       77213 non-null  object
 4   character  67586 non-null  object
 5   role       77213 non-null  object
dtypes: int64(2), object(4)
memory usage: 3.5+ MB
None 

   index  person_id       id             name                character   role
0      0       3748  tm84618   Robert De Niro            Travis Bickle  ACTOR
1      1      14658  tm84618     Jodie Foster            Iris Steensma  ACTOR
2      2       7064  tm84618    Albert Brooks                      Tom  ACTOR
3      3       3739  tm84618    Harvey Keitel  Matthew 'Sport' Higgins  ACTOR
4      4      48933  tm84618  Cybill Shepherd                    Betsy  ACTOR


In [120]:
df5.duplicated().value_counts()

False    77213
dtype: int64

In [121]:
df5['character'].value_counts()

Self                         1667
Himself                      1237
Herself                       444
Self (archive footage)        327
Dancer                        168
                             ... 
Cai Minmin                      1
Toru Miyake                     1
Mattie Ross                     1
Custum officer at airport       1
Young Gallery Man               1
Name: character, Length: 47125, dtype: int64

In [122]:
df5['character'] = df5['character'].str.split("/")
df5 = df5.explode('character')

In [142]:
df5['character'].value_counts()

Self                      1667
Himself                   1237
Herself                    444
Self (archive footage)     327
Dancer                     168
                          ... 
Sigrit Ericksdottir          1
Elle                         1
Doctor (End of Life)         1
Cool Sunnyvale Camper        1
Young Gallery Man            1
Name: character, Length: 48547, dtype: int64

In [143]:
df5 = df5.reset_index()
df5.head()

,level_0,index,person_id,id,name,character,role
0,0,0,3748,tm84618,Robert De Niro,Travis Bickle,ACTOR
1,1,1,14658,tm84618,Jodie Foster,Iris Steensma,ACTOR
2,2,2,7064,tm84618,Albert Brooks,Tom,ACTOR
3,3,3,3739,tm84618,Harvey Keitel,Matthew 'Sport' Higgins,ACTOR
4,4,4,48933,tm84618,Cybill Shepherd,Betsy,ACTOR


In [144]:
df5['index'].value_counts()

18021    19
18019    15
13322    12
90       12
55966    10
         ..
72649     1
68555     1
74702     1
76751     1
0         1
Name: index, Length: 77213, dtype: int64

In [145]:
# Dropping 'index' column and creating a new one after data normalization
# Moving the 'index' column to the front of the dataframe

df5.drop('index', axis=1, inplace=True)

df5['index'] = df5.index
df5.insert(0, 'index', df5.pop('index'))

In [146]:
df5.head()

,index,level_0,person_id,id,name,character,role
0,0,0,3748,tm84618,Robert De Niro,Travis Bickle,ACTOR
1,1,1,14658,tm84618,Jodie Foster,Iris Steensma,ACTOR
2,2,2,7064,tm84618,Albert Brooks,Tom,ACTOR
3,3,3,3739,tm84618,Harvey Keitel,Matthew 'Sport' Higgins,ACTOR
4,4,4,48933,tm84618,Cybill Shepherd,Betsy,ACTOR


In [147]:
df5['index'].value_counts()

2047     1
74417    1
31386    1
29339    1
19100    1
        ..
1386     1
3435     1
13676    1
15725    1
0        1
Name: index, Length: 79010, dtype: int64

In [148]:
df5.drop('level_0', axis=1, inplace=True)

In [149]:
df5.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 79010 entries, 0 to 79009
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   index      79010 non-null  int64 
 1   person_id  79010 non-null  int64 
 2   id         79010 non-null  object
 3   name       79010 non-null  object
 4   character  69383 non-null  object
 5   role       79010 non-null  object
dtypes: int64(2), object(4)
memory usage: 3.6+ MB


In [150]:
df5_val_len = longest_value(df5)

Column: index
Value length: 5
Row possition: 10000 

Column: person_id
Value length: 7
Row possition: 31 

Column: id
Value length: 9
Row possition: 22225 

Column: name
Value length: 73
Row possition: 46763 

Column: character
Value length: 98
Row possition: 63016 

Column: role
Value length: 8
Row possition: 36 



In [151]:
# Saving the dataframe.

df5_csv = save_to_csv(df5)

Path to save location: ../data/output
File name: raw_credits_2nf


## DF6: raw_titles.csv

In [11]:
df6 = open_dataframe('../data/input', 'raw_titles.csv')

In [12]:
df6_info = inspect_dataframe(df6)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5806 entries, 0 to 5805
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   index                 5806 non-null   int64  
 1   id                    5806 non-null   object 
 2   title                 5805 non-null   object 
 3   type                  5806 non-null   object 
 4   release_year          5806 non-null   int64  
 5   age_certification     3196 non-null   object 
 6   runtime               5806 non-null   int64  
 7   genres                5806 non-null   object 
 8   production_countries  5806 non-null   object 
 9   seasons               2047 non-null   float64
 10  imdb_id               5362 non-null   object 
 11  imdb_score            5283 non-null   float64
 12  imdb_votes            5267 non-null   float64
dtypes: float64(3), int64(3), object(7)
memory usage: 589.8+ KB
None 

   index        id                                title  

In [13]:
df6.columns

Index(['index', 'id', 'title', 'type', 'release_year', 'age_certification',
       'runtime', 'genres', 'production_countries', 'seasons', 'imdb_id',
       'imdb_score', 'imdb_votes'],
      dtype='object')

In [14]:
df6.duplicated().value_counts()

False    5806
dtype: int64

In [15]:
df6['production_countries'].unique()

array(["['US']", "['GB']", "['US', 'GB']", "['JP']", "['EG']", "['DE']",
       "['IN']", '[]', "['DZ', 'EG']", "['LB']", "['FR', 'LB', 'CA']",
       "['IN', 'SU']", "['US', 'IT']", "['HK', 'US']", "['GB', 'US']",
       "['AR']", "['KW']", "['EG', 'FR']", "['FR', 'LB']", "['US', 'JP']",
       "['US', 'CA']", "['DE', 'US']", "['US', 'DE']", "['CA', 'US']",
       "['US', 'FR']", "['PE', 'US', 'BR']", "['IT', 'US', 'FR']",
       "['US', 'HK']", "['FR', 'IE', 'GB', 'DE']",
       "['GH', 'GB', 'US', 'BF', 'DE']", "['AR', 'ES']", "['FR']",
       "['MX']", "['AU']", "['CO']", "['BE', 'FR', 'NO', 'LB']",
       "['IT', 'LB', 'BE', 'FR']", "['US', 'FR', 'DE', 'PS']", "['TR']",
       "['CA']", "['GB', 'US', 'BS', 'CZ', 'DE']", "['GB', 'US', 'AU']",
       "['DE', 'ES', 'US', 'FR']", "['GB', 'US', 'AU', 'DE']",
       "['JP', 'US']", "['US', 'IS', 'GB']", "['DK']",
       "['NZ', 'JP', 'US']", "['US', 'CA', 'FR']", "['GB', 'US', 'DE']",
       "['US', 'DE', 'IT', 'GB']", "['CA', 'DE', 'US

In [35]:
# A list of columns where values are list-like values.

df6_cols = list_like_columns(df6)
print(df6_cols)

['genres', 'production_countries']


In [36]:
# Changing the list-like values to proper list values.

df6_list = to_proper_list(df6, df6_cols)

In [37]:
df6_list.head()

,index,id,title,type,release_year,age_certification,runtime,genres,production_countries,seasons,imdb_id,imdb_score,imdb_votes
0,0,ts300399,Five Came Back: The Reference Films,SHOW,1945,TV-MA,48,[documentation],[US],1.0,NaN,NaN,NaN
1,1,tm84618,Taxi Driver,MOVIE,1976,R,113,"[crime, drama]",[US],NaN,tt0075314,8.3,795222.0
2,2,tm127384,Monty Python and the Holy Grail,MOVIE,1975,PG,91,"[comedy, fantasy]",[GB],NaN,tt0071853,8.2,530877.0
3,3,tm70993,Life of Brian,MOVIE,1979,R,94,[comedy],[GB],NaN,tt0079470,8.0,392419.0
4,4,tm190788,The Exorcist,MOVIE,1973,R,133,[horror],[US],NaN,tt0070047,8.1,391942.0


In [63]:
# Normalizing the dataframe to the 2NF.

df6_2nf = copy_and_explode(df6_list, 'genres', 'production_countries')

Enter a name for a normalized dataset: raw_titles


In [16]:
df6_2 = open_dataframe('../data/output', 'raw_titles_2nf.csv')

In [17]:
df6_2.head()

,index,id,title,type,release_year,age_certification,runtime,genres,production_countries,seasons,imdb_id,imdb_score,imdb_votes
0,0,ts300399,Five Came Back: The Reference Films,SHOW,1945,TV-MA,48,documentation,US,1.0,NaN,NaN,NaN
1,1,tm84618,Taxi Driver,MOVIE,1976,R,113,crime,US,NaN,tt0075314,8.3,795222.0
2,1,tm84618,Taxi Driver,MOVIE,1976,R,113,drama,US,NaN,tt0075314,8.3,795222.0
3,2,tm127384,Monty Python and the Holy Grail,MOVIE,1975,PG,91,comedy,GB,NaN,tt0071853,8.2,530877.0
4,2,tm127384,Monty Python and the Holy Grail,MOVIE,1975,PG,91,fantasy,GB,NaN,tt0071853,8.2,530877.0


In [18]:
df6_2['genres'].value_counts()

drama            3437
comedy           2470
thriller         1461
action           1269
romance          1089
documentation    1047
crime            1032
animation         779
fantasy           747
family            741
european          723
scifi             680
horror            452
history           314
music             259
reality           231
war               221
sport             184
western            59
Name: genres, dtype: int64

In [19]:
df6_2[df6_2.loc[:, 'genres'] == 'documentation']

,index,id,title,type,release_year,age_certification,runtime,genres,production_countries,seasons,imdb_id,imdb_score,imdb_votes
0,0,ts300399,Five Came Back: The Reference Films,SHOW,1945,TV-MA,48,documentation,US,1.0,NaN,NaN,NaN
36,13,tm69997,Richard Pryor: Live in Concert,MOVIE,1979,R,78,documentation,US,NaN,tt0079807,8.1,5141.0
45,17,tm14491,The Queen,MOVIE,1968,NaN,68,documentation,US,NaN,tt0183686,7.2,1117.0
46,18,tm89386,Hitler: A Career,MOVIE,1977,PG,150,documentation,DE,NaN,tt0191182,7.5,2416.0
50,19,tm156453,FTA,MOVIE,1972,R,97,documentation,US,NaN,tt0068562,6.2,411.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
17202,5770,ts307816,Korean Cold Noodle Rhapsody,SHOW,2021,TV-PG,49,documentation,KR,1.0,tt15772846,7.3,15.0
17209,5775,tm1146635,Daiki Tsuneta: Tokyo Chaotic,MOVIE,2021,NaN,89,documentation,JP,NaN,tt16781874,7.1,15.0
17213,5777,tm1094060,My Heroes Were Cowboys,MOVIE,2021,PG,23,documentation,US,NaN,tt15084326,NaN,NaN
17214,5778,tm1046969,Myriam Fares: The Journey,MOVIE,2021,NaN,72,documentation,LB,NaN,tt14773250,2.3,153.0


In [20]:
# Replacing 'documentation' to 'documentary'

df6_2 = df6_2.replace({'documentation': 'documentary'})
df6_2[df6_2.loc[:, 'genres'] == 'documentary']

,index,id,title,type,release_year,age_certification,runtime,genres,production_countries,seasons,imdb_id,imdb_score,imdb_votes
0,0,ts300399,Five Came Back: The Reference Films,SHOW,1945,TV-MA,48,documentary,US,1.0,NaN,NaN,NaN
36,13,tm69997,Richard Pryor: Live in Concert,MOVIE,1979,R,78,documentary,US,NaN,tt0079807,8.1,5141.0
45,17,tm14491,The Queen,MOVIE,1968,NaN,68,documentary,US,NaN,tt0183686,7.2,1117.0
46,18,tm89386,Hitler: A Career,MOVIE,1977,PG,150,documentary,DE,NaN,tt0191182,7.5,2416.0
50,19,tm156453,FTA,MOVIE,1972,R,97,documentary,US,NaN,tt0068562,6.2,411.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
17202,5770,ts307816,Korean Cold Noodle Rhapsody,SHOW,2021,TV-PG,49,documentary,KR,1.0,tt15772846,7.3,15.0
17209,5775,tm1146635,Daiki Tsuneta: Tokyo Chaotic,MOVIE,2021,NaN,89,documentary,JP,NaN,tt16781874,7.1,15.0
17213,5777,tm1094060,My Heroes Were Cowboys,MOVIE,2021,PG,23,documentary,US,NaN,tt15084326,NaN,NaN
17214,5778,tm1046969,Myriam Fares: The Journey,MOVIE,2021,NaN,72,documentary,LB,NaN,tt14773250,2.3,153.0


In [21]:
df6_2[df6_2.loc[:, 'production_countries'] == 'Lebanon']

,index,id,title,type,release_year,age_certification,runtime,genres,production_countries,seasons,imdb_id,imdb_score,imdb_votes
2434,649,tm226362,Bosta,MOVIE,2005,NaN,112,war,Lebanon,NaN,tt0497335,6.4,603.0


In [22]:
# Replacing 'Lebanon' to 'documentary'

df6_2 = df6_2.replace({'Lebanon': 'LB'})
df6_2[df6_2.loc[:, 'production_countries'] == 'LB']

,index,id,title,type,release_year,age_certification,runtime,genres,production_countries,seasons,imdb_id,imdb_score,imdb_votes
92,36,tm200475,"Beirut, Oh Beirut",MOVIE,1975,NaN,110,drama,LB,NaN,tt0169599,6.5,106.0
94,37,tm259855,We Are All for the Fatherland,MOVIE,1979,NaN,74,documentary,LB,NaN,tt0170799,6.2,43.0
97,38,tm259684,Whispers,MOVIE,1980,NaN,93,documentary,LB,NaN,tt0170803,7.8,75.0
255,95,tm52274,The Little Wars,MOVIE,1982,NaN,108,drama,LB,NaN,tt0084492,6.3,149.0
257,95,tm52274,The Little Wars,MOVIE,1982,NaN,108,war,LB,NaN,tt0084492,6.3,149.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
14546,4792,ts236850,Children of Adam,SHOW,2020,TV-14,41,action,LB,1.0,tt12434912,7.8,170.0
14547,4792,ts236850,Children of Adam,SHOW,2020,TV-14,41,drama,LB,1.0,tt12434912,7.8,170.0
16729,5543,ts285854,Till Death,SHOW,2021,TV-MA,53,drama,LB,2.0,NaN,NaN,NaN
17214,5778,tm1046969,Myriam Fares: The Journey,MOVIE,2021,NaN,72,documentary,LB,NaN,tt14773250,2.3,153.0


In [23]:
# Changing the index of a dataset.

df6_2.drop('index', axis=1, inplace=True)

df6_2['index'] = df6_2.index
df6_2.insert(0, 'index', df6_2.pop('index'))

In [24]:
df6_2.head()

,index,id,title,type,release_year,age_certification,runtime,genres,production_countries,seasons,imdb_id,imdb_score,imdb_votes
0,0,ts300399,Five Came Back: The Reference Films,SHOW,1945,TV-MA,48,documentary,US,1.0,NaN,NaN,NaN
1,1,tm84618,Taxi Driver,MOVIE,1976,R,113,crime,US,NaN,tt0075314,8.3,795222.0
2,2,tm84618,Taxi Driver,MOVIE,1976,R,113,drama,US,NaN,tt0075314,8.3,795222.0
3,3,tm127384,Monty Python and the Holy Grail,MOVIE,1975,PG,91,comedy,GB,NaN,tt0071853,8.2,530877.0
4,4,tm127384,Monty Python and the Holy Grail,MOVIE,1975,PG,91,fantasy,GB,NaN,tt0071853,8.2,530877.0


In [25]:
df6_2['index'].value_counts()

2047     1
12915    1
17037    1
4743     1
6790     1
        ..
1322     1
7465     1
5416     1
11551    1
0        1
Name: index, Length: 17267, dtype: int64

In [26]:
df6_val_len = longest_value(df6_2)

Column: index
Value length: 5
Row possition: 10000 

Column: id
Value length: 9
Row possition: 2526 

Column: title
Value length: 104
Row possition: 6777 

Column: type
Value length: 5
Row possition: 1 

Column: release_year
Value length: 4
Row possition: 0 

Column: age_certification
Value length: 5
Row possition: 0 

Column: runtime
Value length: 3
Row possition: 1 

Column: genres
Value length: 11
Row possition: 0 

Column: production_countries
Value length: 3
Row possition: 80 

Column: seasons
Value length: 4
Row possition: 158 

Column: imdb_id
Value length: 10
Row possition: 233 

Column: imdb_score
Value length: 3
Row possition: 0 

Column: imdb_votes
Value length: 9
Row possition: 297 



In [27]:
# Saving the dataframe.

df6_csv = save_to_csv(df6_2)

Path to save location: ../data/output
File name: new_raw_titles_2nf


## Files in ../data/output location

In [83]:
output_files = path_to_files()

Enter a path to the files: ../data/output
best_movies_2nf.csv
best_movie_by_year_2nf.csv
best_shows_2nf.csv
best_show_by_year_2nf.csv
raw_credits_2NF.csv
raw_titles_2NF.csv
